# Data collection and preparation:

I collected 15 images with different levels of details. Some had many details, others had overlapping objects, or obvious edges. Others had less details, or less details in some parts of the image, or almost no details. Some had many colours, others had only two.

I tried as much as possible to collect different images that would highlight different effects of different problems, or filters.

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

In [ ]:

image_folder = '/content/original_images'

data = []

for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        img = cv2.imread(os.path.join(image_folder, filename))
        if img is not None:
            # Convert to grayscale
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            data.append({'filename': filename, 'image': gray_img})

original = pd.DataFrame(data)
print(original.head())

In [ ]:

for index, row in original.iterrows():
    img_rgb = cv2.cvtColor(row['image'], cv2.COLOR_BGR2RGB)

    plt.imshow(img_rgb)
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

Now that I have the original images' dataframe I need to create noisy images.
I want to create different dataframes for different types of errors and noise, so when I apply the filters later I can clearly see each filter's effect on different types of noise.

The first dataframe is 'salty', one that contains the same images with salt and pepper noise added to them at different levels.

In [ ]:


def add_salt_pepper_noise(image, prob):
    noisy_image = image.copy()

    num_salt = np.ceil(prob * image.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    noisy_image[coords[0], coords[1]] = 1

    num_pepper = np.ceil(prob * image.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    noisy_image[coords[0], coords[1]] = 0

    return noisy_image

noise_levels = [0.01, 0.08, 0.2]

noisy_data = []

for index, row in original.iterrows():
    for level in noise_levels:
        noisy_img = add_salt_pepper_noise(row['image'], level)
        noisy_data.append({
            'filename': f"{row['filename']}_noise_{level}",
            'image': noisy_img,
            'noise_level': level
        })


salty = pd.DataFrame(noisy_data)
print(salty.head())


In [ ]:

for index, row in salty.iterrows():
    img_rgb = cv2.cvtColor(row['image'], cv2.COLOR_BGR2RGB)

    plt.imshow(img_rgb)
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

So now the salty dataframe contains 3 copies of each original image with 3 different levels of added salt and pepper noise.

Just as before a new dataframe called gaussian_noise is created, but this time a Gaussian noise was added to the images.

In [ ]:

def add_gaussian_noise(image, var, mean=20):
    sigma = var**0.5
    gaussian_noise = np.random.normal(mean, sigma, image.shape)

    noisy_image = image + gaussian_noise

    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

gaussian_variances = [500, 1000, 2000]

noisy_data_gaussian = []

for index, row in original.iterrows():
    for var in gaussian_variances:
        noisy_img = add_gaussian_noise(row['image'], var=var)
        noisy_data_gaussian.append({
            'filename': f"{row['filename']}_gaussian_{var}",
            'image': noisy_img,
            'variance': var
        })

gaussian_noise = pd.DataFrame(noisy_data_gaussian)
print(gaussian_noise.head())


In [ ]:

num_cols = 3


fig, axes = plt.subplots(nrows=len(original), ncols=num_cols, figsize=(20, len(original) * 5))

for i, (index, row) in enumerate(original.iterrows()):
    for j, var in enumerate(gaussian_variances):
        noisy_img = gaussian_noise[
            (gaussian_noise['filename'] == f"{row['filename']}_gaussian_{var}")
        ]['image'].values[0]

        img_rgb = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)

        axes[i, j].imshow(img_rgb)
        axes[i, j].set_title(f"{row['filename']} - Var: {var}")
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()


other types of noise that images could suffer from is the Poisson noise or shot noise, which often appears in images captured by cameras, especially in low-light conditions. image can end up looking grainy and unclean because of it.

In [ ]:
def add_poisson_noise(image):
    noisy_image = np.random.poisson(image).astype(np.uint8)
    noisy_image = np.clip(noisy_image, 0, 255)
    return noisy_image

noisy_data_poisson = []

for index, row in original.iterrows():
    noisy_img = add_poisson_noise(row['image'])
    noisy_data_poisson.append({
        'filename': f"{row['filename']}_poisson",
        'image': noisy_img,
        'noise_type': 'poisson'
    })

noisy_poisson = pd.DataFrame(noisy_data_poisson)
print(noisy_poisson.head())


In [ ]:

for index, row in noisy_poisson.iterrows():
    img_rgb = cv2.cvtColor(row['image'], cv2.COLOR_BGR2RGB)

    plt.imshow(img_rgb)
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

speckl noise:

"Speckle noise is a form of noise that commonly occurs in images acquired through coherent imaging techniques such as ultrasound imaging, synthetic aperture radar (SAR), and laser imaging. Unlike Gaussian noise or salt and pepper noise, which manifest as additive disturbances, speckle noise is multiplicative in nature. This means that it affects the variance of pixel values rather than their mean intensity."

In [ ]:

def generate_speckle_noise(image_shape, mean=0, variance=0.01):
    noise = np.random.normal(mean, variance**0.5, size=image_shape)
    speckle_noise = noise * np.ones(image_shape)
    return speckle_noise

def add_speckle_noise(image, mean=0, variance=0.1):
    speckle_noise = generate_speckle_noise(image.shape, mean=mean, variance=variance)
    noisy_image = image + image * speckle_noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

speckle_variances = [0.05, 0.1, 0.2]

noisy_data_speckle = []

for index, row in original.iterrows():
    for var in speckle_variances:
        noisy_img = add_speckle_noise(row['image'], variance=var)
        noisy_data_speckle.append({
            'filename': f"{row['filename']}_speckle_{var}",
            'image': noisy_img,
            'variance': var
        })

speckle_noise = pd.DataFrame(noisy_data_speckle)
print(speckle_noise.head())


In [ ]:
import matplotlib.pyplot as plt
import cv2

for index, row in speckle_noise.iterrows():
    img_rgb = cv2.cvtColor(row['image'], cv2.COLOR_BGR2RGB)

    plt.imshow(img_rgb)
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

These are some of the many errors and noises that we can find in images in real life, and while some of them may need contrast enhancement, or other image enhancement techniques before using filters, I'm coruse to find if the different types of filters could fix them.

# Applying Filters:
now let us apply different types of filters on these noisy images we generated, and notice how much they're affective in solving their problems, compared to the original images, the ones with no noise.

first let us study simple filter:



In [ ]:
Computational_times = {}
avarage_MSE = {}
avarage_psnr = {}

## 1.   Box filter

### Salt and Pepper noise:

In [ ]:
Computational_times["Salt and Pepper"] = []
avarage_MSE["Salt and Pepper"] = []
avarage_psnr["Salt and Pepper"] = []

In [ ]:

def apply_box_filter(image, ksize=(5, 5)):
    filtered_image = cv2.boxFilter(image, ddepth=-1, ksize=ksize)
    return filtered_image

start_time = time.time()

filtered_data = []

for index, row in salty.iterrows():
    filtered_img = apply_box_filter(row['image'], ksize=(5, 5))
    filtered_data.append({
        'filename': f"{row['filename']}_filtered",
        'image': filtered_img,
    })

end_time = time.time()

total_time = end_time - start_time
Computational_times["Salt and Pepper"].append(total_time)
print(f"Total computational time for applying Box filter: {total_time:.4f} seconds")

filtered_images = pd.DataFrame(filtered_data)
print(filtered_images.head())


In [ ]:

def calculate_mse(original, filtered):
    return np.mean((original - filtered) ** 2)

def calculate_psnr(mse, max_pixel=255.0):
    return 20 * np.log10(max_pixel) - 10 * np.log10(mse) if mse > 0 else float('inf')

counter = 0
mse_sum = 0
psnr_sum = 0

for index, row in original.iterrows():
    original_img = row['image']

    filtered_img_rows = filtered_images[filtered_images['filename'].str.contains(row['filename'])]

    for _, filtered_row in filtered_img_rows.iterrows():
        filtered_img = filtered_row['image']

        mse = calculate_mse(original_img, filtered_img)
        mse_sum += mse
        psnr = calculate_psnr(mse)
        psnr_sum += psnr

        counter +=1

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        axes[0].imshow(original_img, cmap='gray')
        axes[0].set_title(f"Original: {row['filename']}")
        axes[0].axis('off')

        axes[1].imshow(filtered_img, cmap='gray')
        axes[1].set_title(f"MSE: {mse:.2f}, PSNR: {psnr:.2f} dB")
        axes[1].axis('off')

        plt.show()

avrg_mse = mse_sum/counter
avarage_MSE["Salt and Pepper"].append(avrg_mse)
print(f"avarage mse = {avrg_mse}")

avrg_psnr = psnr_sum/counter
avarage_psnr["Salt and Pepper"].append(avrg_psnr)
print(f"avarage psnr = {avrg_psnr}")

As can be seen from the previous output the box filter with kernal size of 5 by 5 didn't perform well on the images, and it did have a somewhat high values of MSE and PSNR, especially on images with median and high level of noise.

However, it indeed completed the work in a short time of only 1.32 seconds.

In [ ]:

def apply_canny_edge(image, threshold1=100, threshold2=200):
    edges = cv2.Canny(image, threshold1, threshold2)
    return edges

canny_data = []

for index, row in filtered_images.iterrows():
    canny_edges = apply_canny_edge(row['image'], threshold1=100, threshold2=200)
    canny_data.append({
        'filename': f"{row['filename']}_canny",
        'image': canny_edges
    })

canny_edges_df = pd.DataFrame(canny_data)

for index, row in canny_edges_df.iterrows():
    plt.imshow(row['image'], cmap='gray')
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()


Canny edge detection helped visualize the box filter short coming, as it detected much noise as edges in 2 thirds of the images (images with median and high levels of noise).

These output are almost the same in all images regardless of the amount of details in them. while of course the ones with higher level of details suffering the most. while being more visible with images with a wide area of low information.

let's try the same but this time with a bigger kernel, let's say an 11 by 11 one. over the douple of the last one.

In [ ]:

def apply_box_filter(image, ksize=(5, 5)):
    filtered_image = cv2.boxFilter(image, ddepth=-1, ksize=ksize)
    return filtered_image

start_time = time.time()

filtered_data = []

for index, row in salty.iterrows():
    filtered_img = apply_box_filter(row['image'], ksize=(11, 11))
    filtered_data.append({
        'filename': f"{row['filename']}_filtered",
        'image': filtered_img,
    })

end_time = time.time()

total_time = end_time - start_time
Computational_times["Salt and Pepper"].append(total_time)
print(f"Total computational time for applying Box filter: {total_time:.4f} seconds")

filtered_images = pd.DataFrame(filtered_data)

In [ ]:

def calculate_mse(original, filtered):
    return np.mean((original - filtered) ** 2)

def calculate_psnr(mse, max_pixel=255.0):
    return 20 * np.log10(max_pixel) - 10 * np.log10(mse) if mse > 0 else float('inf')

counter = 0
mse_sum = 0
psnr_sum = 0

for index, row in original.iterrows():
    original_img = row['image']

    filtered_img_rows = filtered_images[filtered_images['filename'].str.contains(row['filename'])]

    for _, filtered_row in filtered_img_rows.iterrows():
        filtered_img = filtered_row['image']

        mse = calculate_mse(original_img, filtered_img)
        mse_sum += mse
        psnr = calculate_psnr(mse)
        psnr_sum += psnr

        counter +=1

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        axes[0].imshow(original_img, cmap='gray')
        axes[0].set_title(f"Original: {row['filename']}")
        axes[0].axis('off')

        axes[1].imshow(filtered_img, cmap='gray')
        axes[1].set_title(f"MSE: {mse:.2f}, PSNR: {psnr:.2f} dB")
        axes[1].axis('off')

        plt.show()

avrg_mse = mse_sum/counter
avarage_MSE["Salt and Pepper"].append(avrg_mse)
print(f"avarage mse = {avrg_mse}")

avrg_psnr = psnr_sum/counter
avarage_psnr["Salt and Pepper"].append(avrg_psnr)
print(f"avarage psnr = {avrg_psnr}")

As seen from these measures, and from the visualizations, the big kernel didn't perform bitter. as the nature of the salt and pepper noise could not be solved by reducing the gaps between neighboring pixels.

And with a bigger kernel the box filter method would calculate a not neccesary close values to each other, due to the nature of this noise.

As in a big kernel many pepper noise could exist with salt noise, they could cancel each other, or worsen the effect.

In [ ]:

def apply_canny_edge(image, threshold1=100, threshold2=200):
    edges = cv2.Canny(image, threshold1, threshold2)
    return edges

canny_data = []

for index, row in filtered_images.iterrows():
    canny_edges = apply_canny_edge(row['image'], threshold1=100, threshold2=200)
    canny_data.append({
        'filename': f"{row['filename']}_canny",
        'image': canny_edges
    })

canny_edges_df = pd.DataFrame(canny_data)

for index, row in canny_edges_df.iterrows():
    plt.imshow(row['image'], cmap='gray')
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()


the bigger kernel blurred the thin and weak edges with the background. Hence why Canny no longer could detect them.

### Gaussian noise:

In [ ]:
Computational_times["Gaussian"] = []
avarage_MSE["Gaussian"] = []
avarage_psnr["Gaussian"] = []

In [ ]:
start_time = time.time()

filtered_data = []

for index, row in gaussian_noise.iterrows():
    filtered_img = apply_box_filter(row['image'], ksize=(5, 5))
    filtered_data.append({
        'filename': f"{row['filename']}_filtered",
        'image': filtered_img,
    })

end_time = time.time()

total_time = end_time - start_time
Computational_times["Gaussian"].append(total_time)
print(f"Total computational time for applying Box filter: {total_time:.4f} seconds")

filtered_images = pd.DataFrame(filtered_data)

In [ ]:
counter = 0
mse_sum = 0
psnr_sum = 0

for index, row in original.iterrows():
    original_img = row['image']

    filtered_img_rows = filtered_images[filtered_images['filename'].str.contains(row['filename'])]

    for _, filtered_row in filtered_img_rows.iterrows():
        filtered_img = filtered_row['image']

        mse = calculate_mse(original_img, filtered_img)
        mse_sum += mse
        psnr = calculate_psnr(mse)
        psnr_sum += psnr

        counter +=1

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        axes[0].imshow(original_img, cmap='gray')
        axes[0].set_title(f"Original: {row['filename']}")
        axes[0].axis('off')

        axes[1].imshow(filtered_img, cmap='gray')
        axes[1].set_title(f"MSE: {mse:.2f}, PSNR: {psnr:.2f} dB")
        axes[1].axis('off')

        plt.show()

avrg_mse = mse_sum/counter
avarage_MSE["Gaussian"].append(avrg_mse)
print(f"avarage mse = {avrg_mse}")

avrg_psnr = psnr_sum/counter
avarage_psnr["Gaussian"].append(avrg_psnr)
print(f"avarage psnr = {avrg_psnr}")

The images this time look smoother and more visually appealing, as the box filter works well with Gaussian noise. It works on bringing the pixels values closer to each other (to their neighbors), and Gaussian noise tend to be gradual. However, it doesn't work on returning the pixel to its original value, hence why the average MSE and PSNR are still high.

In [ ]:
canny_data = []

for index, row in filtered_images.iterrows():
    canny_edges = apply_canny_edge(row['image'], threshold1=100, threshold2=200)
    canny_data.append({
        'filename': f"{row['filename']}_canny",
        'image': canny_edges
    })

canny_edges_df = pd.DataFrame(canny_data)

for index, row in canny_edges_df.iterrows():
    plt.imshow(row['image'], cmap='gray')
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

As could be seen, much of the fine details got too blurred to be detected by Canny as edges. As the Gaussian noise mush them, and the box filter blurs them even more.

In [ ]:
start_time = time.time()

filtered_data = []

for index, row in gaussian_noise.iterrows():
    filtered_img = apply_box_filter(row['image'], ksize=(11, 11))
    filtered_data.append({
        'filename': f"{row['filename']}_filtered",
        'image': filtered_img,
    })

end_time = time.time()

total_time = end_time - start_time
Computational_times["Gaussian"].append(total_time)
print(f"Total computational time for applying Box filter: {total_time:.4f} seconds")

filtered_images = pd.DataFrame(filtered_data)

In [ ]:
counter = 0
mse_sum = 0
psnr_sum = 0

for index, row in original.iterrows():
    original_img = row['image']

    filtered_img_rows = filtered_images[filtered_images['filename'].str.contains(row['filename'])]

    for _, filtered_row in filtered_img_rows.iterrows():
        filtered_img = filtered_row['image']

        mse = calculate_mse(original_img, filtered_img)
        mse_sum += mse
        psnr = calculate_psnr(mse)
        psnr_sum += psnr

        counter +=1

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        axes[0].imshow(original_img, cmap='gray')
        axes[0].set_title(f"Original: {row['filename']}")
        axes[0].axis('off')

        axes[1].imshow(filtered_img, cmap='gray')
        axes[1].set_title(f"MSE: {mse:.2f}, PSNR: {psnr:.2f} dB")
        axes[1].axis('off')

        plt.show()

avrg_mse = mse_sum/counter
avarage_MSE["Gaussian"].append(avrg_mse)
print(f"avarage mse = {avrg_mse}")

avrg_psnr = psnr_sum/counter
avarage_psnr["Gaussian"].append(avrg_psnr)
print(f"avarage psnr = {avrg_psnr}")

In [ ]:
canny_data = []

for index, row in filtered_images.iterrows():
    canny_edges = apply_canny_edge(row['image'], threshold1=100, threshold2=200)
    canny_data.append({
        'filename': f"{row['filename']}_canny",
        'image': canny_edges
    })

canny_edges_df = pd.DataFrame(canny_data)

for index, row in canny_edges_df.iterrows():
    plt.imshow(row['image'], cmap='gray')
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

Same as with salt and pepper noise, with a bigger kernel the images got smoother, yet the details got more blurred, leading to edges detection suffering even more. Now only very strong and broad edges are detected. The image skeleton is unobserved in most images, especially images with many details

### Poisson noise:

In [ ]:
Computational_times["Poisson"] = []
avarage_MSE["Poisson"] = []
avarage_psnr["Poisson"] = []

In [ ]:
start_time = time.time()

filtered_data = []

for index, row in noisy_poisson.iterrows():
    filtered_img = apply_box_filter(row['image'], ksize=(5, 5))
    filtered_data.append({
        'filename': f"{row['filename']}_filtered",
        'image': filtered_img,
    })

end_time = time.time()

total_time = end_time - start_time
Computational_times["Poisson"].append(total_time)
print(f"Total computational time for applying Box filter: {total_time:.4f} seconds")

filtered_images = pd.DataFrame(filtered_data)

In [ ]:
counter = 0
mse_sum = 0
psnr_sum = 0

for index, row in original.iterrows():
    original_img = row['image']

    filtered_img_rows = filtered_images[filtered_images['filename'].str.contains(row['filename'])]

    for _, filtered_row in filtered_img_rows.iterrows():
        filtered_img = filtered_row['image']

        mse = calculate_mse(original_img, filtered_img)
        mse_sum += mse
        psnr = calculate_psnr(mse)
        psnr_sum += psnr

        counter +=1

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        axes[0].imshow(original_img, cmap='gray')
        axes[0].set_title(f"Original: {row['filename']}")
        axes[0].axis('off')

        axes[1].imshow(filtered_img, cmap='gray')
        axes[1].set_title(f"MSE: {mse:.2f}, PSNR: {psnr:.2f} dB")
        axes[1].axis('off')

        plt.show()

avrg_mse = mse_sum/counter
avarage_MSE["Poisson"].append(avrg_mse)
print(f"avarage mse = {avrg_mse}")

avrg_psnr = psnr_sum/counter
avarage_psnr["Poisson"].append(avrg_psnr)
print(f"avarage psnr = {avrg_psnr}")

While the filter smoothed the image, still some of the darkened parts of the images because of the noise remained visible. And even pulled the filter to darken the image more in low-detailed images.

In [ ]:
canny_data = []

for index, row in filtered_images.iterrows():
    canny_edges = apply_canny_edge(row['image'], threshold1=100, threshold2=200)
    canny_data.append({
        'filename': f"{row['filename']}_canny",
        'image': canny_edges
    })

canny_edges_df = pd.DataFrame(canny_data)

for index, row in canny_edges_df.iterrows():
    plt.imshow(row['image'], cmap='gray')
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

The low-detailed images suffered from noise being detected as edges in them. While the high-detailed image suffered from its fine details being too blurred to be detected.

That is due to the box calculating higher new values for the pixels affected and being surrounded by noise in low information areas. And blurring the edges with noise in high information areas.

In [ ]:
start_time = time.time()

filtered_data = []

for index, row in noisy_poisson.iterrows():
    filtered_img = apply_box_filter(row['image'], ksize=(11, 11))
    filtered_data.append({
        'filename': f"{row['filename']}_filtered",
        'image': filtered_img,
    })

end_time = time.time()

total_time = end_time - start_time
Computational_times["Poisson"].append(total_time)
print(f"Total computational time for applying Box filter: {total_time:.4f} seconds")

filtered_images = pd.DataFrame(filtered_data)

In [ ]:
counter = 0
mse_sum = 0
psnr_sum = 0

for index, row in original.iterrows():
    original_img = row['image']

    filtered_img_rows = filtered_images[filtered_images['filename'].str.contains(row['filename'])]

    for _, filtered_row in filtered_img_rows.iterrows():
        filtered_img = filtered_row['image']

        mse = calculate_mse(original_img, filtered_img)
        mse_sum += mse
        psnr = calculate_psnr(mse)
        psnr_sum += psnr

        counter +=1

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        axes[0].imshow(original_img, cmap='gray')
        axes[0].set_title(f"Original: {row['filename']}")
        axes[0].axis('off')

        axes[1].imshow(filtered_img, cmap='gray')
        axes[1].set_title(f"MSE: {mse:.2f}, PSNR: {psnr:.2f} dB")
        axes[1].axis('off')

        plt.show()

avrg_mse = mse_sum/counter
avarage_MSE["Poisson"].append(avrg_mse)
print(f"avarage mse = {avrg_mse}")

avrg_psnr = psnr_sum/counter
avarage_psnr["Poisson"].append(avrg_psnr)
print(f"avarage psnr = {avrg_psnr}")

In [ ]:
canny_data = []

for index, row in filtered_images.iterrows():
    canny_edges = apply_canny_edge(row['image'], threshold1=100, threshold2=200)
    canny_data.append({
        'filename': f"{row['filename']}_canny",
        'image': canny_edges
    })

canny_edges_df = pd.DataFrame(canny_data)

for index, row in canny_edges_df.iterrows():
    plt.imshow(row['image'], cmap='gray')
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

The bigger kernel worsened the edges problem, with now even low detailed images with strong and broad edges got too blurred to be detected.

### speckle noise:

In [ ]:
Computational_times["Speckle"] = []
avarage_MSE["Speckle"] = []
avarage_psnr["Speckle"] = []

In [ ]:
start_time = time.time()

filtered_data = []

for index, row in speckle_noise.iterrows():
    filtered_img = apply_box_filter(row['image'], ksize=(5, 5))
    filtered_data.append({
        'filename': f"{row['filename']}_filtered",
        'image': filtered_img,
    })

end_time = time.time()

total_time = end_time - start_time
Computational_times["Speckle"].append(total_time)
print(f"Total computational time for applying Box filter: {total_time:.4f} seconds")

filtered_images = pd.DataFrame(filtered_data)

In [ ]:
counter = 0
mse_sum = 0
psnr_sum = 0

for index, row in original.iterrows():
    original_img = row['image']

    filtered_img_rows = filtered_images[filtered_images['filename'].str.contains(row['filename'])]

    for _, filtered_row in filtered_img_rows.iterrows():
        filtered_img = filtered_row['image']

        mse = calculate_mse(original_img, filtered_img)
        mse_sum += mse
        psnr = calculate_psnr(mse)
        psnr_sum += psnr

        counter +=1

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        axes[0].imshow(original_img, cmap='gray')
        axes[0].set_title(f"Original: {row['filename']}")
        axes[0].axis('off')

        axes[1].imshow(filtered_img, cmap='gray')
        axes[1].set_title(f"MSE: {mse:.2f}, PSNR: {psnr:.2f} dB")
        axes[1].axis('off')

        plt.show()

avrg_mse = mse_sum/counter
avarage_MSE["Speckle"].append(avrg_mse)
print(f"avarage mse = {avrg_mse}")

avrg_psnr = psnr_sum/counter
avarage_psnr["Speckle"].append(avrg_psnr)
print(f"avarage psnr = {avrg_psnr}")

In [ ]:
canny_data = []

for index, row in filtered_images.iterrows():
    canny_edges = apply_canny_edge(row['image'], threshold1=100, threshold2=200)
    canny_data.append({
        'filename': f"{row['filename']}_canny",
        'image': canny_edges
    })

canny_edges_df = pd.DataFrame(canny_data)

for index, row in canny_edges_df.iterrows():
    plt.imshow(row['image'], cmap='gray')
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

The filter smoothed the images with low and median noise levels, while not effecting the edges too much. However, it couldn't smooth the noise in high noise level images enough. That's why Canny detected the noise as edges in these images.

That may hint at using a bigger kernel to solve such a problem. Let's try this out.

In [ ]:
start_time = time.time()

filtered_data = []

for index, row in speckle_noise.iterrows():
    filtered_img = apply_box_filter(row['image'], ksize=(11, 11))
    filtered_data.append({
        'filename': f"{row['filename']}_filtered",
        'image': filtered_img,
    })

end_time = time.time()

total_time = end_time - start_time
Computational_times["Speckle"].append(total_time)
print(f"Total computational time for applying Box filter: {total_time:.4f} seconds")

filtered_images = pd.DataFrame(filtered_data)

In [ ]:
counter = 0
mse_sum = 0
psnr_sum = 0

for index, row in original.iterrows():
    original_img = row['image']

    filtered_img_rows = filtered_images[filtered_images['filename'].str.contains(row['filename'])]

    for _, filtered_row in filtered_img_rows.iterrows():
        filtered_img = filtered_row['image']

        mse = calculate_mse(original_img, filtered_img)
        mse_sum += mse
        psnr = calculate_psnr(mse)
        psnr_sum += psnr

        counter +=1

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        axes[0].imshow(original_img, cmap='gray')
        axes[0].set_title(f"Original: {row['filename']}")
        axes[0].axis('off')

        axes[1].imshow(filtered_img, cmap='gray')
        axes[1].set_title(f"MSE: {mse:.2f}, PSNR: {psnr:.2f} dB")
        axes[1].axis('off')

        plt.show()

avrg_mse = mse_sum/counter
avarage_MSE["Speckle"].append(avrg_mse)
print(f"avarage mse = {avrg_mse}")

avrg_psnr = psnr_sum/counter
avarage_psnr["Speckle"].append(avrg_psnr)
print(f"avarage psnr = {avrg_psnr}")

In [ ]:
canny_data = []

for index, row in filtered_images.iterrows():
    canny_edges = apply_canny_edge(row['image'], threshold1=100, threshold2=200)
    canny_data.append({
        'filename': f"{row['filename']}_canny",
        'image': canny_edges
    })

canny_edges_df = pd.DataFrame(canny_data)

for index, row in canny_edges_df.iterrows():
    plt.imshow(row['image'], cmap='gray')
    plt.title(row['filename'])
    plt.axis('off')
    plt.show()

While a bigger kernel did smooth the images more, this led to more edges being too blurred to be detected.

Now Canny couldn't draw the skeleton for all high-detailed images with most of their edges being originally thin and weak. And even low-detailed images with originaly strong and broad edges, were not detected correctly.


All while MSE and PSNR are still high, and the pictures aren't that much visually appealing. In my opinion the trade off wasn't worth it. And going to higher kernels isn't an option if edge detection is the next step in the application.

### Box filter results:

In [ ]:
comp_times_df = pd.DataFrame(Computational_times, index=['5x5 Kernel', '11x11 Kernel']).T
mse_df = pd.DataFrame(avarage_MSE, index=['5x5 Kernel', '11x11 Kernel']).T
psnr_df = pd.DataFrame(avarage_psnr, index=['5x5 Kernel', '11x11 Kernel']).T

In [ ]:
print("Computational Times:")
comp_times_df.style.set_caption("Computational Times").set_table_styles([{'selector': 'th', 'props': [('border', '1px solid black')]}])

It's very clear that the bigger kernel took a longer computational time in all cases. Still, the box filter as a whole took a very short time compared to other filters. That's due to it being the simplest.

In [ ]:
print("\nAverage MSEs:")
mse_df.style.set_caption("Average MSEs").set_table_styles([{'selector': 'th', 'props': [('border', '1px solid black')]}])


On the other hand, in all cases a bigger kernel worsens the average MSE (mean square error). That is because the box filter calculates the pixel's new value according to its neighbors' values. A bigger kernel has a higher chance of containing more noisy pixels, which will lead the new value to lean further from the one in the original image.

In [ ]:
print("\nAverage PSNRs:")
psnr_df.style.set_caption("Average PSNRs").set_table_styles([{'selector': 'th', 'props': [('border', '1px solid black')]}])


The PSNR (peak signal to noise ratio) stays almost the same in all cases. The box filter smoothes the overall image, bringing the pixels' values closer to their neighbors. Yet as the kernel's weights are distributed equally, this smoothing effect is not that much influential. Even with a bigger kernel.